In [1]:
import math
import numpy as np
import pandas as pd
import scipy
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt

import scipy.stats
from scipy import stats, optimize
from scipy.stats import genextreme

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler

sn.set(rc={'figure.figsize':(21, 9)})
%matplotlib notebook

In [2]:
data = pd.read_csv('combine_partial_lag.csv')

In [3]:
data

,East_12,lag_east_12,13,14,15,16,17,18,19,20,...,49.2,50.2,51.2,52.2,53.2,54.2,55.2,56.2,57.2,58.2
0,-0.017703,-0.016553,0.021267,0.042634,0.082478,0.130230,0.139420,0.141490,0.124130,0.14460,...,0.002730,0.001201,-0.001766,0.005172,0.004519,0.006129,0.004212,0.004655,0.001408,0.003451
1,-0.001151,-0.028014,0.027178,0.053463,0.090675,0.120980,0.137650,0.148710,0.132350,0.13501,...,0.002280,-0.000927,-0.002955,0.004738,0.002229,0.001678,-0.002080,0.002313,-0.000923,0.004295
2,0.026863,0.010981,0.039995,0.089400,0.115810,0.121480,0.137530,0.156670,0.142270,0.13988,...,0.001380,0.000830,-0.001670,0.001650,-0.000317,-0.003118,-0.006825,-0.001222,-0.003579,-0.002215
3,0.015882,0.039388,0.027085,0.082073,0.115360,0.133080,0.152590,0.178490,0.173680,0.18923,...,-0.000976,0.001851,-0.000283,-0.001392,-0.001417,-0.005652,-0.007379,-0.001470,-0.001904,-0.003790
4,-0.023506,0.011090,-0.003510,0.034058,0.056934,0.098202,0.130510,0.160280,0.180060,0.21011,...,-0.000047,0.000326,-0.001703,-0.001424,-0.002720,-0.006904,-0.006104,-0.001440,-0.001904,-0.004034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.011548,0.025305,0.019385,0.036875,0.049536,0.066969,0.083730,0.103420,0.125480,0.16158,...,-0.002378,-0.001360,-0.004021,-0.001477,-0.000846,-0.001404,-0.001987,-0.002977,-0.001452,0.001027
1996,-0.036853,-0.048127,-0.021727,0.008923,0.020413,0.029443,0.030310,0.043885,0.069490,0.10237,...,-0.001412,-0.002070,-0.005016,-0.001092,0.000303,-0.000318,-0.000741,-0.001565,-0.002887,0.002378
1997,0.011274,-0.042309,0.009199,0.041144,0.048578,0.051872,0.041825,0.053639,0.073084,0.10071,...,-0.003319,-0.004697,-0.005705,-0.001817,-0.002971,-0.002289,-0.003073,-0.003446,-0.002566,-0.000146
1998,0.053583,0.015161,0.057363,0.069681,0.084376,0.079812,0.066351,0.097193,0.127860,0.15697,...,-0.005805,-0.002575,-0.002744,-0.001202,-0.003122,-0.003818,-0.002937,-0.002672,0.000917,-0.000873


In [4]:
data_values = data.values
features = data_values[:,1:]
values = data_values[:,0]

In [5]:
features[0]

array([-0.0165525 ,  0.021267  ,  0.042634  ,  0.082478  ,  0.13023   ,
        0.13942   ,  0.14149   ,  0.12413   ,  0.1446    ,  0.14609   ,
        0.15797   ,  0.15817   ,  0.18298   ,  0.20063   ,  0.19354   ,
        0.17362   ,  0.14189   ,  0.12908   ,  0.099288  ,  0.087275  ,
        0.094222  ,  0.075285  ,  0.092737  ,  0.099984  ,  0.097626  ,
        0.10753   ,  0.138     ,  0.15982   ,  0.19522   ,  0.22058   ,
        0.19663   ,  0.19659   ,  0.19191   ,  0.22582   ,  0.22947   ,
        0.24498   ,  0.25199   ,  0.26311   ,  0.22778   ,  0.21771   ,
        0.21851   ,  0.22598   ,  0.20083   ,  0.23436   ,  0.27743   ,
        0.35043   ,  0.39029   ,  0.07903   ,  0.12183   ,  0.16657   ,
        0.20121   ,  0.27049   ,  0.33337   ,  0.3659    ,  0.37105   ,
        0.35508   ,  0.33496   ,  0.34246   ,  0.33889   ,  0.33552   ,
        0.31189   ,  0.31892   ,  0.31178   ,  0.3275    ,  0.32089   ,
        0.32349   ,  0.34008   ,  0.34135   ,  0.34157   ,  0.33

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100,
       subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
       min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None,
       init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, 
       warm_start=False, presort='auto', validation_fraction=0.1, tol=0.0001)

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

clf2 = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [7]:
pred1 = []
pred2 = []

In [8]:
n = 1599
i = 0
for j in range(n, len(features)-1):
    temp_training_features = features[i:j]
    temp_training_values = values[i:j]
    temp_training_features.reshape(-1,1)
    temp_training_values.reshape(-1,1)
    clf.fit(temp_training_features, temp_training_values, sample_weight=None)
    #clf2.fit(temp_training_features, temp_training_values, sample_weight=None)
    # m = 1
    test_features = features[j+1]
    test_features.reshape(-1,1)
    temp_pred1 = clf.predict([test_features])
    #temp_pred2 = clf2.predict([test_features])
    # n = n+1
    i = i+1
    pred1.append(temp_pred1)
    #pred2.append(temp_pred2)

In [9]:
pred1

[array([-0.01485082]),
 array([-0.011868]),
 array([-0.01044712]),
 array([-0.0120276]),
 array([-0.02262809]),
 array([-0.03107958]),
 array([-0.00503877]),
 array([0.04491879]),
 array([0.06306484]),
 array([0.03006844]),
 array([0.03779413]),
 array([0.07724908]),
 array([0.0914923]),
 array([0.09293347]),
 array([0.11799318]),
 array([0.1125447]),
 array([0.09522085]),
 array([0.11274471]),
 array([0.10614743]),
 array([0.0750955]),
 array([0.04935339]),
 array([0.03637043]),
 array([0.06468914]),
 array([0.09026751]),
 array([0.08909432]),
 array([0.06906955]),
 array([0.06938496]),
 array([0.06569884]),
 array([0.04549802]),
 array([0.06075688]),
 array([0.09419334]),
 array([0.11270036]),
 array([0.09508987]),
 array([0.06690479]),
 array([0.0652142]),
 array([0.07580206]),
 array([0.09508764]),
 array([0.09085449]),
 array([0.07511533]),
 array([0.08289596]),
 array([0.0994437]),
 array([0.12344359]),
 array([0.04172073]),
 array([-0.06653248]),
 array([-0.05618195]),
 array([0

In [10]:
len(pred1)

251

In [11]:
#pred2

In [12]:
#len(pred2)

In [13]:
test_values = values[1600:2000]

In [14]:
len(test_values)

251

In [15]:
from sklearn.metrics import r2_score
r2_score(test_values, pred1)

0.9810562767301239

In [16]:
#r2_score(test_values, pred2)

In [17]:
#0.9858030592317613#1
#0.9857613191494161#2
#0.9853963491546662#3
#0.9853407753411109#4
#0.9854888222455#5
#0.9854890415802979#8
#0.9842227305866622#24
#0.9815789240020949#100
#0.9812406718954232#150